In [1]:
import pandas as pd
import nltk
import string
from nltk.stem import PorterStemmer

w2v = {}
df = pd.read_csv('dataset_lyrics.csv')

nltk.download('stopwords')
stop_words = set(nltk.corpus.stopwords.words('english'))
docs_without_punctuation = []
ps = PorterStemmer()

docs_without_stopwords = df['lyrics'].apply(lambda x: ' '.join([w for w in x.split() if w not in (stop_words)]))
for row in docs_without_stopwords:
  docs_without_punctuation.append(row.translate(str.maketrans('','', string.punctuation)))
stemmed = [[ps.stem(w) for w in row.split(" ")] for row in docs_without_punctuation]



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
from gensim.models import Word2Vec
model1 = Word2Vec(stemmed, min_count = 1,size = 10, window = 5, sg=0)
model1.wv.vocab

{'hold': <gensim.models.keyedvectors.Vocab at 0x7fd919e54ee0>,
 'time': <gensim.models.keyedvectors.Vocab at 0x7fd8faa300d0>,
 'feel': <gensim.models.keyedvectors.Vocab at 0x7fd8fb638c70>,
 'break': <gensim.models.keyedvectors.Vocab at 0x7fd8fb638d60>,
 'untru': <gensim.models.keyedvectors.Vocab at 0x7fd8faa4d4f0>,
 'convinc': <gensim.models.keyedvectors.Vocab at 0x7fd8faa4d520>,
 'speak': <gensim.models.keyedvectors.Vocab at 0x7fd8faa4d580>,
 'voic': <gensim.models.keyedvectors.Vocab at 0x7fd8faa4d5e0>,
 'tear': <gensim.models.keyedvectors.Vocab at 0x7fd8faa4d640>,
 'tri': <gensim.models.keyedvectors.Vocab at 0x7fd8faa4d6a0>,
 'hurt': <gensim.models.keyedvectors.Vocab at 0x7fd8faa4d700>,
 'forgiv': <gensim.models.keyedvectors.Vocab at 0x7fd8faa4d760>,
 'okay': <gensim.models.keyedvectors.Vocab at 0x7fd8faa4d7c0>,
 'play': <gensim.models.keyedvectors.Vocab at 0x7fd8faa4d820>,
 'string': <gensim.models.keyedvectors.Vocab at 0x7fd8faa4d880>,
 'heart': <gensim.models.keyedvectors.Vocab at

In [67]:
print(model1.wv['fuck']) # so bekommt man vec zu einem word

[ 2.6880612   0.42962563 -1.1897742   0.3774033  -5.1299424   4.4137287
  0.08863656 -1.3925571   2.4479744  -2.9419377 ]


In [91]:
import numpy as np
def avg_pool(doc):
    pool = np.array([0.0]*10)
    
    for idx, word in enumerate(doc):
      vec = model1.wv[word]
      pool += vec[0]

    pool = pool / len(doc)

        
    return pool

def cosine_sim(a, b):
  dot = np.dot(a, b)
  aNorm = np.linalg.norm(a)
  bNorm = np.linalg.norm(b)

  denom = (aNorm * bNorm)

  if denom > 0.0:
    return dot / denom
  elif dot == 0.0:
    return 0.0

In [95]:
track_to_wv = dict()
track_to_lyrics = dict()
#df['track_name']
#df['lyrics']

test = 100
cnt = 0
for index, row in df.iterrows():
  docs_without_punctuation = []
  docs_without_stopwords = [w for w in row['lyrics'].split() if w not in (stop_words)]
  docs_without_punctuation = [w for w in docs_without_stopwords if w not in (string.punctuation)]

  stemmed = [[ps.stem(w) for w in roww.split(" ")] for roww in docs_without_punctuation]

  track_to_lyrics[str(row['artist_name']) + " " + str(row['track_name']) + " " + str(row['release_date'])] = stemmed
  track_to_wv[str(row['artist_name']) + " " + str(row['track_name']) + " " + str(row['release_date'])] = avg_pool(stemmed)

  if cnt == test:
    break
  cnt = cnt + 1

print(track_to_wv)

{'mukesh mohabbat bhi jhoothi 1950': array([-1.77028683, -0.69404095,  0.18478055, -1.46944811, -0.75359081,
        2.36865113,  2.27526551,  0.82930831,  3.42772331, -0.71461148]), 'frankie laine i believe 1950': array([-2.77027606, -2.43559211, -0.34617033, -0.48572491, -0.453473  ,
        1.55660926,  1.58817366, -0.05169161,  2.81678545,  0.28698053]), 'johnnie ray cry 1950': array([-1.18704142, -1.43818731, -0.28468971, -2.15222747,  0.49554741,
        0.77186405,  1.56838365,  0.59981934,  3.14964553, -0.42449171]), 'pérez prado patricia 1950': array([-0.76072543, -0.85019948, -1.34371035, -0.89310883, -0.01274846,
        0.4262154 ,  1.37718605,  0.26767075,  2.82763242, -0.43626468]), 'giorgos papadopoulos apopse eida oneiro 1950': array([-2.62461649, -1.94795888, -0.26345293, -0.66748361, -0.69482765,
        0.36419906,  2.42857255,  1.40691439,  2.91020512,  0.48460304]), 'perry como round and round (with mitchell ayres and his orchestra & the ray charles singers) 1950':

In [96]:
user_query = "hello i love you"

user_query_without_stopwords = []
user_query_without_punctuation = []
user_query_without_stopwords = [w for w in user_query.split() if w not in (stop_words)]
user_query_without_punctuation = [w for w in user_query_without_stopwords if w not in (string.punctuation)]

user_query_stemmed = [[ps.stem(w) for w in roww.split(" ")] for roww in user_query_without_punctuation]

print(user_query_stemmed)
print(avg_pool(user_query_stemmed))

[['hello'], ['love']]
[-2.08549887 -0.58359259 -2.89519131 -3.725788    1.50278896  1.21593612
  1.79405272 -1.50135636  4.57406056 -0.07916918]


In [116]:
import torch
from torch import nn

# Create model
device = "cpu"

# Define model
class LyricsModel(nn.Module):
    def __init__(self):
        super(LyricsModel, self).__init__()
        self.linear = torch.nn.Sequential(
            torch.nn.Linear(10, 10),
            torch.nn.ReLU(),
            torch.nn.Linear(10, 1)
            
        )

    def forward(self, inputs_):
        x = self.linear(inputs_)
        return x

model = LyricsModel().to(device)
model = model.float()

# Your code
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3) #todo
loss_fn = nn.MSELoss(reduction='sum') #todo

In [141]:
def train(data, model, loss_fn, optimizer):
  model.train()

  for key, item in data.items():
    # predict output using one hot encoding of word

    pred = model(torch.tensor(item).float())

    target = cosine_sim(avg_pool(user_query_stemmed), item)

    print(pred)
    print(torch.tensor([target]).float())

    # Compute prediction error
    loss = loss_fn(pred, torch.tensor([target]).float())

    print(loss)

    # Backpropagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [ ]:
def test(data, model, loss_fn, optimizer):
  return

In [144]:
optimizer = torch.optim.Adagrad(model.parameters(), lr=0.025)
loss_fn = torch.nn.BCEWithLogitsLoss()

epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(track_to_wv, model, loss_fn, optimizer)
    #test(test_dataloader, model, loss_fn, optimizer)
print("Done!")

Epoch 1
-------------------------------
tensor([-0.1806], grad_fn=<AddBackward0>)
tensor([0.7080])
tensor(0.7348, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor([0.3777], grad_fn=<AddBackward0>)
tensor([0.6863])
tensor(0.6405, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor([0.2804], grad_fn=<AddBackward0>)
tensor([0.8529])
tensor(0.6040, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor([0.4623], grad_fn=<AddBackward0>)
tensor([0.8711])
tensor(0.5481, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor([0.8069], grad_fn=<AddBackward0>)
tensor([0.6154])
tensor(0.6793, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor([0.2777], grad_fn=<AddBackward0>)
tensor([0.3646])
tensor(0.7404, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor([0.5102], grad_fn=<AddBackward0>)
tensor([0.7889])
tensor(0.5779, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor([0.7348], grad_fn=<AddBackward0>)
tensor([0.7682])
tensor(0.5621, grad_fn=<BinaryCrossEntropyWithL